# Job Shop Scheduling Problem
### Informe 1 - Trabajo de Investigación

Integrantes: Sebastian Orellana  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;  Francisco Russeau

## Problema
Se consideran n trabajos. Cada trabajo se compone de un conjunto de m operaciones, las cuales deben procesarse en un orden específico (restricciones de precedencia). El procesamiento de cada operación se asocia a una máquina específica. En un momento dado solo se puede procesar una operación de un trabajo. Se intenta minimizar el tiempo total de la programación que procesa todos los trabajos.

Seda, M. (2007). Mathematical Models of Flow Shop and Job Shop Scheduling
Problems. World Academy of Science, Engineering and Technology, International
Journal of Mathematical, Computational, Physical, Electrical and Computer Engineering, 1, 307-312.
https://citeseerx.ist.psu.edu/viewdoc/summary?doi=10.1.1.193.2697

# Modelo

Se definen los módulos y el solver a utilizar para optimizar el problema

In [ ]:
using JuMP
using GLPK

Se crea una instancia del solver para optimizar el modelo.

In [ ]:
jssp = Model(GLPK.Optimizer)

## Constantes
Se define la matriz de tiempos de procesamiento para cada trabajo j asociado a la máquina m y también la matriz de secuenciación de cada trabajo j asociado a la máquina m.

In [ ]:
# Matriz de tiempos de procesamiento del trabajo j para la máquina m
 T = [5  7  10
     9  5  3
     5  8  2
     2  7  4
     8  8  8]

# Matriz de secuenciación del trabajo j para la máquina m
P = [2  1  3
     1  2  3
     3  2  1
     2  1  3
     3  1  2]

Luego se definen otras constantes que serán usadas más adelante cuando se definan las variables y restricciones del modelo.

In [ ]:
J,M = size(T)
V = sum(T)
n = 1:J;
m = 1:M;

## Variables de desición 
A continuación se definen las variables de desición que se utilizarán en el modelo.

In [ ]:
#################################################################################################################
#################################         VARIABLES DE DECISIÓN           #######################################

# x(i,j) -> instante de inicio del trabajo j en la máquina i
@variable(jssp, x[j = 1:J, i = 1:M] ≥ 0)

# z(k,j,i) = 1 -> si el trabajo j precede al trabajo k en la máquina i
@variable(jssp, z[k = 1:J, j = 1:J, i = 1:M], Bin)

# Cmax -> instante de término del n-ésimo trabajo en la m-ésima máquina (makespan)
@variable(jssp, Cmax ≥ 0)

## Función Objetivo
Luego, se define la función objetivo que se desea optimizar. En este caso corresponde a Minimizar el makespan, es decir, minimizar el tiempo que demoran en conjunto todas las máquinas en ejecutar todas las tareas.

In [ ]:
#################################################################################################################
#################################        FUNCIÓN OBJETIVO           #############################################

# Min z = Cmax
@objective(jssp, Min, Cmax)

## Restricciones
Luego, se definen las restricciones a utilizar en el modelo y que acotarán el conjunto de desiciones.

In [ ]:
#################################################################################################################
####################################         RESTRICCIONES         ##############################################

# x(i,j) >= 0,           ∀ j = 1:J, i = 1:M
@constraint(jssp, [i = 1:M, j = 1:J], x[j,i] ≥ 0)

# x(P[h,j], j) >= x(P[h-1,j], j) + T(P[h-1,j], j),            ∀ j = 1:J, h = 2:M
@constraint(jssp, [i = 1:M-1, j = 1:J], x[j,P[j,i+1]] ≥ x[j,P[j,i]] + T[j,P[j,i]])

# x(i,j) ≥ x(i,k) + T(i,k) − V · z(i,j,k),                 ∀ j e k 1:J, j < k, i ∈ M
@constraint(jssp, [i = 1:M, j = 1:J, k = 1:J; j ≠ k], x[j,i] ≥ x[k,i] + T[k,i] - V * z[k,j,i])

# x(i,k) ≥ x(i,j) + T(i,j) − V · (1 − z(i,j,k)),             ∀ j, k ∈ J, j < k, i ∈ M
@constraint(jssp, [i = 1:M, j = 1:J, k = 1:J; j ≠ k], x[k,i] ≥ x[j,i] + T[j,i] - V * (1 - z[k,j,i]))

# Cmax ≥ x(P[m,j], j) + T(P[m,j], j),                        ∀ j ∈ J
@constraint(jssp, [j = 1:J], Cmax ≥ x[j,P[j,M]] + T[j,P[j,M]])

# Optimizar el modelo
Por último, se optimiza el modelo utilizando el solver GLPK, el cual para problemas enteros utiliza el método ramificación y acotamiento.

In [ ]:
#################################################################################################################
###################################         SOLUCIÓN           ##################################################

# Optimizar el modelo
optimize!(jssp)

# Valor de la función objetivo en el óptimo
objective_value(jssp)

## Variables de la solución óptima

In [ ]:
println("instante de inicio del trabajo j en la máquina i")
for j in 1:J
    for i in 1:M
        println("Variable X",j,i," = ",value(x[j,i]))
    end
end

println("")
println("Si el trabajo j precede al trabajo k en la máquina i")
for k in 1:J
    for j in 1:J
        for i in 1:M
            println("Variable Z",k,j,i," = ",value(z[k,j,i])) 
        end
    end
end

## Otras matrices que se utilizaron

Para realizar pruebas sobre el modelo definido y registrar los tiempos de ejecución se utilizaron diferentes matrices con diferentes tamaños, es decir varíaba la cantidad de máquinas y trabajos que se debían procesar por cada una. A contniuación, se muestran las otras matrices que fueron utilizadas para las pruebas.

### Matriz de 5x4

In [ ]:
#5x4
T = [25 75 75 76 
     67  5 11 11 
     22 98  8 35 
     99 42  2 35 
     50  5 59 71]

P = [1 3 4 2
     2 1 3 4
     1 2 4 3
     3 1 2 4
     4 2 1 3]

# Constantes
J,M = size(T)
V = sum(T)
n = 1:J;
m = 1:M;

### Matriz de 5x5

In [ ]:
T = [25 75 75 76 12 
     67  5 11 11 11
     22 98  8 35 9
     99 42  2 35 19
     50  5 59 71 13]

P = [1 3 4 2 5
     2 1 3 5 4
     1 2 5 4 3
     3 5 1 2 4
     4 2 5 1 3]

#Constantes
J,M = size(T)
V = sum(T)
n = 1:J;
m = 1:M;

### Matriz de 6x5

In [ ]:
T = [15 85 15 66 22  
     57 25 26 9 11
     23 68 8 35 9
     76 32 12 35 19
     41 15 23 62 13
     32 19 28 45 29]

P = [1 3 4 2 5
     2 1 3 5 4
     2 1 3 4 5
     2 4 5 3 1
     1 3 2 5 4
     5 4 2 1 3]

#Constantes
J,M = size(T)
V = sum(T)
n = 1:J;
m = 1:M;

### Matriz de 6x6

In [ ]:
#6x6
T = [15 45 15 36 22 56  
     57 25 26 9 11 43
     23 28 8 35 9 26
     56 32 12 35 19 32
     41 15 23 62 13 26
     32 19 28 45 29 43]

P = [1 3 4 2 5 6
     2 1 3 6 5 4
     2 1 6 3 4 5
     2 4 5 6 3 1
     6 1 3 2 5 4
     5 6 4 2 1 3]

# Constantes
J,M = size(T)
V = sum(T)
n = 1:J;
m = 1:M;

### Matriz de 7x6

In [ ]:
#7x6
T = [15 45 15 36 22 56  
     57 25 26 9 11 43
     23 28 8 35 9 26
     56 32 12 35 19 32
     41 15 23 62 13 26
     32 19 28 45 29 43
     12 5 6 23 15 21]

P = [1 3 4 2 5 6
     2 1 3 6 5 4
     2 1 6 3 4 5
     2 4 5 6 3 1
     6 1 3 2 5 4
     5 6 4 2 1 3
     4 3 6 5 2 1]

# Constantes
J,M = size(T)
V = sum(T)
n = 1:J;
m = 1:M;

### Matriz de 15x15

In [ ]:
T = [ 94 66 10 53 26 15 65 82 10 27 93 92 96 70 83
      74 31 88 51 57 78  8  7 91 79 18 51 18 99 33
       4 82 40 86 50 54 21  6 54 68 82 20 39 35 68
      73 23 30 30 53 94 58 93 32 91 30 56 27 92  9
      78 23 21 60 36 29 95 99 79 76 93 42 52 42 96
      29 61 88 70 16 31 65 83 78 26 50 87 62 14 30
      18 75 20  4 91 68 19 54 85 73 43 24 37 87 66
      32 52  9 49 61 35 99 62  6 62  7 80  3 57  7
      85 30 96 91 13 87 82 83 78 56 85  8 66 88 15
       5 59 30 60 41 17 66 89 78 88 69 45 82  6 13
      90 27  1  8 91 80 89 49 32 28 90 93  6 35 73
      47 43 75  8 51  3 84 34 28 60 69 45 67 58 87
      65 62 97 20 31 33 33 77 50 80 48 90 75 96 44
      28 21 51 75 17 89 59 56 63 18 17 30 16  7 35
      57 16 42 34 37 26 68 73  5  8 12 87 83 20 97]

P = [ 7 13  5  8  4  3 11 12  9 15 10 14  6  1  2
      5  6  8 15 14  9 12 10  7 11  1  4 13  2  3
      2  9 10 13  7 12 14  6  1  3  8 11  5  4 15
      6  3 10  7 11  1 14  5  8 15 12  9 13  2  4
      8  9  7 11  5 10  3 15 13  6  2 14 12  1  4
      6  4 13 14 12  5 15  8  3  2 11  1 10  7  9
     13  4  8  9 15  7  2 12  5  6  3 11  1 14 10
     12  6  1  8 13 14 15  2  3  9  5  4 10  7 11
     11 12  7 15  1  2  3  6 13  5  9  8 10 14  4
      7 12 10  3  9  1 14  4 11  8  2 13 15  5  6
      5  8 14  1  6 13  7  9 15 11  4  2 12 10  3
      3 15  1 13  7 11  8  6  9 10 14  2  4 12  5
      6  9 11  3  4  7 10  1 14  5  2 12 13  8 15
      9 15  5 14  6  7 10  2 13  8 12 11  4  3  1
     11  9 13  7  5  2 14 15 12  1  8  4  3 10  6]

# Constantes
J,M = size(T)
V = sum(T)
n = 1:J;
m = 1:M;